## Parse: Encode search as document vector

In [1]:
import pandas as pd
from sklearn.externals import joblib
transformer = joblib.load('../project-06/data/vectorizer.pkl')

In [3]:
from os import chdir
chdir('../project-06/')

In [4]:
import lib.database_module as db
import lib.encoding_module as enc

In [5]:
db.connect_to_postgres()

Connected to server joshuacook.me.


(<connection object at 0x116903b00; dsn: 'user=russell password=xxxxxx income laid fairly' host=joshuacook.me port=5432 dbname=project_6', closed: 0>,
 <cursor object at 0x11565cde0; closed: 0>)

In [6]:
page_vectors=db.select_all_page_vectors()

Connected to server joshuacook.me.


In [7]:
indices=[tup[0] for tup in page_vectors]
vectors=[tup[1] for tup in page_vectors]

In [8]:
page_vectors_df=pd.DataFrame(vectors, index=indices)

## Mine: Select document vectors for all pages from database

In [9]:
page_vectors_df.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
4341789,0.092992,-0.040985,0.006952,0.000243,0.001681,0.004999,0.003430,0.003523,-0.005129,-0.001911,...,-0.021003,-0.039528,0.027716,0.021033,-0.034768,-0.022294,0.031537,-0.009710,-0.010331,0.025944
48201744,0.021301,0.027559,-0.003917,-0.010877,0.013055,0.000934,0.000128,0.010317,0.001444,-0.009792,...,-0.055040,0.000110,0.057817,0.005156,0.058477,0.008807,0.043527,-0.011693,0.064738,0.051296
2514975,0.124983,0.153025,0.026264,-0.257191,-0.210183,0.074847,0.003862,-0.009666,-0.065705,-0.280972,...,-0.022703,0.015894,-0.002476,0.004992,-0.006653,-0.017394,-0.008525,0.017188,-0.001596,0.000120
35135520,0.191357,-0.039003,-0.006018,0.006389,-0.001106,-0.004748,0.015829,0.005612,0.011846,0.028248,...,0.000578,-0.004204,0.030916,0.010484,-0.012454,0.026050,0.038744,0.015332,-0.008053,0.009506
27303975,0.034292,0.039499,-0.006095,-0.015156,0.026112,-0.039749,-0.081440,-0.036546,0.005076,0.018881,...,0.030056,-0.026965,-0.002426,0.042282,0.024082,0.004498,-0.009517,-0.005315,-0.003784,-0.010202


## Model: Find five most similar documents based on document vectors

In [10]:
from sklearn.neighbors import NearestNeighbors

In [11]:
from lib.encoding_module import get_searchterm_vector, get_page_vector


In [12]:
search_term='bicep'

In [13]:
search_term2=get_searchterm_vector(transformer,search_term)
search_term2

{'bicep': array([[  7.26502477e-05,   2.14800270e-04,  -6.92913722e-05,
          -9.42956153e-05,   2.58611997e-04,  -3.85704897e-04,
          -6.76760878e-04,  -4.20531138e-04,   2.89618956e-05,
           9.66837029e-05,  -1.21135597e-04,  -9.27411692e-06,
          -1.79036923e-04,   5.55867389e-05,  -1.24528466e-04,
          -6.23710307e-06,  -8.84230097e-05,   7.39473527e-05,
           1.03230723e-05,   1.17047000e-05,   5.93108605e-05,
          -1.01682271e-05,   6.08115167e-05,  -5.39187512e-06,
          -5.14327047e-04,  -2.62617537e-04,   1.04773402e-04,
           9.67539234e-05,   9.61321176e-05,   2.57810575e-04,
          -1.21537371e-04,  -4.25659177e-04,   3.05313418e-04,
          -8.21872931e-05,   1.79669356e-04,  -1.64947092e-04,
          -2.20635309e-05,  -3.68490777e-05,  -6.38089426e-05,
           1.72747783e-04,  -9.39380859e-05,   6.34919405e-05,
           1.09541850e-04,  -1.02670786e-04,  -5.47554085e-05,
          -1.66285302e-04,  -1.08382406e-04,  

In [14]:
search_term2.values()[0][0]

array([  7.26502477e-05,   2.14800270e-04,  -6.92913722e-05,
        -9.42956153e-05,   2.58611997e-04,  -3.85704897e-04,
        -6.76760878e-04,  -4.20531138e-04,   2.89618956e-05,
         9.66837029e-05,  -1.21135597e-04,  -9.27411692e-06,
        -1.79036923e-04,   5.55867389e-05,  -1.24528466e-04,
        -6.23710307e-06,  -8.84230097e-05,   7.39473527e-05,
         1.03230723e-05,   1.17047000e-05,   5.93108605e-05,
        -1.01682271e-05,   6.08115167e-05,  -5.39187512e-06,
        -5.14327047e-04,  -2.62617537e-04,   1.04773402e-04,
         9.67539234e-05,   9.61321176e-05,   2.57810575e-04,
        -1.21537371e-04,  -4.25659177e-04,   3.05313418e-04,
        -8.21872931e-05,   1.79669356e-04,  -1.64947092e-04,
        -2.20635309e-05,  -3.68490777e-05,  -6.38089426e-05,
         1.72747783e-04,  -9.39380859e-05,   6.34919405e-05,
         1.09541850e-04,  -1.02670786e-04,  -5.47554085e-05,
        -1.66285302e-04,  -1.08382406e-04,  -5.88771051e-05,
        -1.09509928e-04,

In [15]:
import numpy as np


In [16]:
this_NN=NearestNeighbors()
this_NN.fit(page_vectors_df)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [17]:
this_NN.kneighbors(search_term2.values()[0][0])

/Users/russellsasaki/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(array([[ 0.23923308,  0.34949915,  0.36504524,  0.37490897,  0.37745513]]),
 array([[1172,  247,  795,  483,  566]]))

In [23]:
print page_vectors_df.index[1172]
print page_vectors_df.index[247]
print page_vectors_df.index[795]
print page_vectors_df.index[483]
print page_vectors_df.index[566]

8181732
18007329
2904185
13846997
38485157


## Mine: Retrieve text for the five most similar documents

In [24]:
from lib.wiki_module import query_page

In [33]:
query_page('8181732')['text']

u'Lithuanian Player of the Year is a football award. Winners 1965: Petras Glodenis ( \u017dalgiris Vilnius) 1966: Gintautas Kal\u0117dinskas ( \u017dalgiris Vilnius) 1967: Stanislovas Ramelis ( \u017dalgiris Vilnius) 1968: Stanislovas Ramelis ( \u017dalgiris Vilnius) 1969: Juzefas Jurgelevi\u010dius ( \u017dalgiris Vilnius) 1970: Romualdas Ju\u0161ka ( \u017dalgiris Vilnius) 1971: Benjaminas Zelkevi\u010dius ( \u017dalgiris Vilnius) 1972: Benjaminas Zelkevi\u010dius ( \u017dalgiris Vilnius) 1973: Petras Glodenis ( \u017dalgiris Vilnius) 1974: Algirdas \u017dilinskas ( \u017dalgiris Vilnius) 1975: Vytautas Dirmeikis ( \u017dalgiris Vilnius) 1976: Eugenijus Riabovas ( \u017dalgiris Vilnius) 1977: Eugenijus Riabovas ( \u017dalgiris Vilnius) 1978: Eugenijus Riabovas ( \u017dalgiris Vilnius) 1979: Stanislovas Danisevi\u010dius ( \u017dalgiris Vilnius) 1980: Juzefas Jurgelevi\u010dius ( \u017dalgiris Vilnius) 1981: Vytautas Dirmeikis ( \u017dalgiris Vilnius) 1982: Sigitas Jakubauskas ( \u017

In [34]:
query_page('18007329')['text']

u'Karate terms come almost entirely from Japanese. The following terms are not exclusive to karate. They appear during its study and practice, varying depending on style and school. Karate terms include: C Chito-ryu D Dan \u2013 Dojo G Gi \u2013 Goju-ryu - H Hajime \u2013 Heian K Karate \u2013 Kata \u2013 Kihon \u2013 Kohai \u2013 Kumite \u2013 Ky\u016b O Osu R Rei S Senpai \u2013 Sensei \u2013 Shihan \u2013 Shotokan \u2013 S\u014dke W Waza Z Zanshin See also References External links'

In [35]:
query_page('2904185')['text']

u'Simit (Turkish: simit, Arabic: \u0633\u0645\u064a\u0637\u200e, translit. sam\u012bt\u02e4\u200e, Persian: \u0633\u06cc\u0645\u06cc\u062a\u200e, translit. s\u012bm\u012bt\u02e4\u200e, Bulgarian: \u0441\u0438\u043c\u0438\u0442  simit ), gevrek (Turkish: gevrek, Bulgarian: \u0433\u0435\u0432\u0440\u0435\u043a, Macedonian: \u0453\u0435\u0432\u0440\u0435\u043a, Serbian: \u0452\u0435\u0432\u0440\u0435\u043a), bokegh (Armenian: \u0562\u0578\u056f\u0565\u0572), or koulouri (Greek: \u03ba\u03bf\u03c5\u03bb\u03bf\u03cd\u03c1\u03b9) is a circular bread, typically encrusted with sesame seeds or, less commonly, poppy, flax or sunflower seeds, found across the cuisines of the former Ottoman Empire, and the Middle East. Simits size, crunch, chewiness, and other characteristics vary slightly by region. In \u0130zmir, simit is known as gevrek ("crisp"), although it is very similar to the Istanbul variety. Simits in Ankara are smaller and crisper than those of other cities. Simits in Istanbul are made

In [36]:
query_page('13846997')['text']

u'The Covin is a replica kit car of the Porsche 911 Turbo created by Tim Cook and Nick Vincent in the early 1980s. The name Covin came about from CO (Cook) and VIN (Vincent) giving us COVIN Performance Mouldings. Early models of the Covin were based on a shortened Beetle floorpan running gear but later used its own Covin chassis and VW Type 3 running gear. The company was sold in the 1990s to DAX and later moved to new owners GPC and was relocated to County Galway in Ireland where unfortunately up to now the Covin has not been produced again. The Covin came in 3 body styles all based on the Turbo model. These were, Flatnose and Coupe, both were also available in a convertible version. There is also known to be only one narrow body Covin ever made. Covin also produced a few 356 Speedster models but these are rather rare. Some Covin units even run real Porsche 911 engines. The Covin Club The Covin Owners Club is run by Darren Parker (Club Technical Adviser) & Club Secretary Michael Dykes

In [37]:
query_page('38485157')['text']

u'The exercise d\u01b0\u1ee1ng sinh or D\u01b0\u1ee1ng Sinh (compare Chinese Yang-Sheng \u990a\u751f) is a form of partly indigenous breathing and yoga exercise similar to Tai Chi popularized in Vietnam by the historian and political activist Nguy\u1ec5n Kh\u1eafc Vi\u1ec7n. Vi\u1ec7n had been trained as a medical doctor in the field of womens and childrens psychotherapy. When he himself was given three years to live he turned to practice of traditional breathing exercises. However although the name "d\u01b0\u1ee1ng sinh" was popularised by Vi\u1ec7n in T\u1eeb sinh l\xfd \u0111\u1ebfn d\u01b0\u1ee1ng sinh and other books, the idea of "d\u01b0\u1ee1ng sinh" is well known to village people anyway as a compliment to thu\u1ed1c Nam herbal medicine. References'